# Homology of simplicial complexes

Define the homology of simplicial complexes, generalizing the homology of graphs.

In [ ]:
import numpy as np
from itertools import combinations

## Directed simplicial complexes

A **directed simplicial complex** is a simplicial complex together with a total order on the vertices of each simplex satisfying the following consistency condition: if $v \lt_{\sigma} w$ in a simplex $\sigma$ then $v \lt_\rho w$ for any face $\rho$ of $\sigma$.

Any simplicial complex can be made into a directed simplicial complex by, for example, choosing a total order of all its vertices.

We will model directed simplicial complexes as ``sets`` of ``tuples`` of vertices.

In [ ]:
def add_disimplex(dicomplex, disimplex):
    """
    Add a directed simplex to the directed complex along with all its directed faces.

    Parameters:
        simplicial_complex (set): A set of tuples.
        simplex (iterable): An iterable of vertices defining the simplex.
    """
    disimplex = tuple(disimplex)

    # Adds all subtuples of the simplex (excluding the empty tuple)
    for r in range(1, len(disimplex) + 1):
        for diface in combinations(disimplex, r):
            dicomplex.add(tuple(diface))

### Example

The torus as a directed simplicial complex.
Notice that the vertices are already naturally ordered below.

In [ ]:
torus_top_simplices = [
    [0, 1, 2], [1, 2, 4], [1, 3, 4], [1, 3, 6], [0, 1, 5], [1, 5, 6],
    [2, 3, 5], [2, 4, 5], [2, 3, 6], [0, 2, 6], [0, 3, 4], [0, 3, 5],
    [4, 5, 6], [0, 4, 6]
]

torus = set()
for simplex in torus_top_simplices:
    add_disimplex(torus, simplex)

## Homology

Let us fix a ground field $\Bbbk$ over which all vector spaces will be considered.

Let $X$ be a directed simplicial complex and $X_n$ its set of $n$-dimensional simplices for some non-negative integer $n$.

Define $C_n$ as the vector spaces generated by $X_n$.
Elements in this vector space are referred to as **(degree) $n$-chains**.


Let $\partial_n \colon C_{n} \to C_{n-1}$ be the linear map, referred to as the **boundary map**, defined on basis elements by:

$$
\partial(v_0, \dots, v_n) = \sum_{i=0}^n (-1)^i (v_0, \dots, \widehat{v}_i, \dots, v_n).
$$

Chains in the subvector space $\operatorname{img}(\partial_{n+1})$ are referred to as **(degree) $n$-boundaries**, whereas chains in the subvector space $\operatorname{ker}(\partial_n)$ are referred to as **(degree) $n$-cycles**.

The **(degree) $n$-homology** of $X$ with coefficients in $\Bbbk$, denoted $H_n(X; \Bbbk)$, is the quotient of the $n$-cycles by the $n$-boundaries.
Explicitly,

$$
H_n = \operatorname{ker}(\partial_n) / \operatorname{img}(\partial_{n+1}).
$$

Notice that this generalizes the homology of graphs, since by convention, $\partial_0 = 0$.

---

#### Challenge 1

Show that every boundary is a cycle, i.e., that $\operatorname{img}(\partial_{n+1}) \subseteq \operatorname{ker}(\partial_n)$ or, equivalently, that $\partial_n \circ \partial_{n+1} = 0$.

---

#### Challenge 2

Show that the homology of a simplicial complex (is up to isomorphism) independent of any chosen directed structure.

---

## Boundary matrix

Consider a directed simplicial complex $X$ and a non-negative integer $n$.

Let $B_n$ be the matrix representation of $\partial_n$ with respect to the canonical bases of $X_n$ and $X_{n-1}$ ordered arbitrarily.

Since
$$\operatorname{dim}\mathrm{ker}(\partial_n) = \operatorname{null}(B_n)$$
and
$$\operatorname{dim}\mathrm{img}(\partial_{n+1}) = \operatorname{rank}(B_{n+1})$$
we have
$$
\operatorname{dim} H_n = \operatorname{null}(B_n) - \operatorname{rank}(B_{n+1}).
$$

We refer to $\operatorname{dim} H_n$ as the **(degree) $n$-betti** number of $X$, denoting it $\beta_n$.

The boundary matrices depend on the specific linear orders chosen for the bases.
However, their rank and nullities remains invariant under different orderings, so we generally do not need to be overly concerned with the ordering.

---

#### Challenge 2

Use the rank-nullity theorem to prove that the Euler characteristic of a simplicial complex satisfies:
$$
\chi = \sum_{i \geq 0} (-1)^i\beta_i.
$$

---

In [ ]:
def boundary_matrix(simplicial_complex, n):
    """
    Compute the boundary matrix of degree n for a simplicial complex.

    Parameters:
    - simplicial_complex: set of frozensets, where each frozenset represents a simplex.
    - n: integer, the dimension of simplices for the columns of the matrix.

    Returns:
    - B: numpy array, the boundary matrix of size (num_(n-1)_simplices) x (num_n_simplices).
    - n_simplices: list of n-simplices (columns of B).
    - n_minus_1_simplices: list of (n-1)-simplices (rows of B).
    """
    # Extract simplices of dimension n and (n-1) and canonically order them
    n_simplices = [s for s in simplicial_complex if len(s) == n + 1]
    n_simplices = sorted(n_simplices)
    n_minus_1_simplices = [s for s in simplicial_complex if len(s) == n]
    n_minus_1_simplices = sorted(n_minus_1_simplices)

    # Create an index mapping for (n-1)-simplices
    n_minus_1_simplex_index = {s: i for i, s in enumerate(n_minus_1_simplices)}

    # Initialize the boundary matrix
    B = np.zeros((len(n_minus_1_simplices), len(n_simplices)), dtype=int)

    # Fill the boundary matrix
    if n != 0:
        for j, simplex in enumerate(n_simplices):
            for pos, face in enumerate(combinations(simplex, n)):
                sign = (-1)**(pos + n)  # Use consistent behaviour of combinations
                B[n_minus_1_simplex_index[face], j] = sign

    return B, n_simplices, n_minus_1_simplices


Let us inspect the boundary matrices of the torus.

In [ ]:
n = 2
B, n_simplices, n_minus_1_simplices = boundary_matrix(torus, n)
print(f"{n}-boundary matrix:\n")
print(B)

2-boundary matrix:

[[ 1  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [-1  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 -1  0  1  0  0  0  0  0  0  0  0]
 [ 0 -1  0  0 -1  0  0  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0 -1  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  1  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1 -1  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 -1 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  1  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0 -1  0 -1  0]
 [ 0  0  1  0  0  0  0  0  0  0  0 -1  0  0]
 [ 0  0  0  1  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  1]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  0 -1]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  0  1]]


Let us check that the every boundary is a cycle.
This is equivalent to show that $\partial_n \circ \partial_{n+1} = 0$, or, in terms of matrices, that $B_nB_{n+1} = 0$.

In [ ]:
n = 1
B1, _, _ = boundary_matrix(torus, n)
B2, _, _ = boundary_matrix(torus, n+1)
print(np.dot(B1, B2))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


## Homology of surfaces and other complexes

Let us now compute the homology of some complexes.
We will work over the field with two elements.
We first need to implement a column reduction function for binary matrices.
Let us start by recalling from, the last notebook, the directed simplicial complexes we will work with.

In [ ]:
names = ["cicle", "sphere", "torus", "Klein", "rp2", "s2vs1"]

all_top_simplices = []

all_top_simplices.append([
    [0,1], [0,2], [1,2]
])

all_top_simplices.append([
    [0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]
])

all_top_simplices.append([
    [0, 1, 2], [1, 2, 4], [1, 3, 4], [1, 3, 6],
    [0, 1, 5], [1, 5, 6], [2, 3, 5], [2, 4, 5],
    [2, 3, 6], [0, 2, 6], [0, 3, 4], [0, 3, 5],
    [4, 5, 6], [0, 4, 6]
])

all_top_simplices.append([
    [2, 3, 7], [1, 2, 3], [1, 3, 5], [1, 5, 7],
    [1, 4, 7], [2, 4, 6], [1, 2, 6], [1, 6, 0],
    [1, 4, 0], [2, 4, 0], [3, 4, 7], [3, 4, 6],
    [3, 5, 6], [5, 6, 0], [2, 5, 0], [2, 5, 7]
])

all_top_simplices.append([
    [0, 1, 2], [0, 2, 3], [0, 1, 5], [0, 4, 5],
    [0, 3, 4], [1, 2, 4], [1, 3, 4], [1, 3, 5],
    [2, 3, 5], [2, 4, 5]
])

all_top_simplices.append([
    [0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3],
    [3, 4], [3, 5], [4, 5]
])

These top simplices will be converted into directed simplicial complexes using the following downward closure method.

In [ ]:
def dicomplex_from_top_simplices(top_simplices):
    """
    construct the directed simplicial complex from its top directed simplices.

    """
    dicomplex = set()
    for simplex in top_simplices:
        add_disimplex(dicomplex, simplex)
    return dicomplex

Let us recall the reduction implementation over mod 2 coefficients.

In [ ]:
def _pivot(column):
    """
    Find the index of the first nonzero entry in a column.

    Args:
        column (numpy.ndarray): A 1D NumPy array representing a column of the matrix.

    Returns:
        int: The index of the first nonzero entry.
             Returns -1 if the column consists entirely of zeros.
    """
    nonzero_indices = np.where(column)[0]
    return nonzero_indices[0] if nonzero_indices.size > 0 else -1

def reduce_matrix(matrix):
    """
    Perform column reduction on a binary matrix over the field ℤ/2ℤ (mod 2).

    The reduction follows a variant of Gaussian elimination tailored for binary matrices.
    Columns with the same leading (pivot) entry are XOR-ed to eliminate redundancy.

    Args:
        matrix (numpy.ndarray): A 2D boolean NumPy array representing the matrix to be reduced.

    Returns:
        numpy.ndarray: A reduced binary matrix, where each column has a unique pivot (if possible).
    """
    col_num = matrix.shape[1]  # Number of columns in the matrix
    reduced = np.array(matrix, dtype=bool)  # Ensure the matrix is binary (mod 2)
    i = -1

    while i < col_num - 1:  # Iterate over columns
        i += 1
        if _pivot(reduced[:, i]) == -1:  # Skip zero columns
            continue

        j = i
        while j < col_num - 1:  # Iterate over all later columns
            j += 1
            piv_i = _pivot(reduced[:, i])
            piv_j = _pivot(reduced[:, j])

            if piv_i == piv_j:  # If two columns share the same pivot, reduce the later one
                reduced[:, j] = np.logical_xor(reduced[:, j], reduced[:, i])
                i = -1  # Restart outer while loop to check all columns again

    return reduced

To compute the betti numbers we read off the rank and nullity of the reduced matrices.

In [ ]:
def betti_number(complex, n):
    """
    Compute the Betti number of a simplicial complex at a given dimension
    using mod 2 coefficients

    Parameters:
    - complex: set of frozensets, where each frozenset represents a simplex.
    - n: integer, the degree of the computed homology.

    Returns:
    - betti: integer, the degree n Betti number.
    """
    B1, _, _ = boundary_matrix(complex, n)
    reduced_B1 = reduce_matrix(B1)
    nullity = np.sum(np.all(reduced_B1 == 0, axis=0))

    B2, _, _ = boundary_matrix(complex, n+1)
    reduced_B2 = reduce_matrix(B2)
    rank = np.sum(np.any(reduced_B2, axis=0))

    return nullity - rank

Let us now compute the Betti numbers of our complexes in degrees 0, 1, 2.

In [ ]:
print("Betti  | 0 | 1 | 2")
print("-"*18)
for name, top_simplices in zip(names, all_top_simplices):
    dicomplex = dicomplex_from_top_simplices(top_simplices)
    b0 = betti_number(dicomplex, 0)
    b1 = betti_number(dicomplex, 1)
    b2 = betti_number(dicomplex, 2)
    print(f"{name:<6} | {b0} | {b1} | {b2}")

Betti  | 0 | 1 | 2
------------------
cicle  | 1 | 1 | 0
sphere | 1 | 0 | 1
torus  | 1 | 2 | 1
Klein  | 1 | 2 | 1
rp2    | 1 | 1 | 1
s2vs1  | 1 | 1 | 1


### Explanations

**Connectedness:**  
The first Betti number $\beta_0$ is always 1, indicating that each space is connected.  

**Circle vs. sphere:**  
- The circle has $\beta_1 = 1$, reflecting its single nontrivial loop, and $\beta_2 = 0$, as it has no 2-dimensional holes.  
- The sphere has $\beta_1 = 0$ since it has no nontrivial 1-cycles, but $\beta_2 = 1$, representing its enclosed 2-dimensional cavity.  

**Torus and Klein bottle:**  
- Both have $\beta_1 = 2$, indicating two independent 1-dimensional cycles.  
- They also share $\beta_2 = 1$, corresponding to the enclosed 2-dimensional region.  

**Real projective plane vs. wedge of a sphere and a circle:**  
- The real projective plane $\mathbb{RP}^2$ has a single 1-dimensional cycle $\beta_1 = 1$ and a single 2-dimensional cavity $\beta_2 = 1$.  
- The wedge of a sphere and a circle $S^2 \vee S^1$ has the same Betti numbers: $\beta_1 = 1$ from the circle and $\beta_2 = 1$ from the sphere.


---

**Challenge 3**

Show that betti numbers are additive with respect to disjoint union.

---